In [2]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
import numpy as np
import os
directory = 'C:\\Users\\giorg\\OneDrive\\Υπολογιστής\\DL Project'
os.chdir(directory)


c:\Users\giorg\OneDrive\Υπολογιστής\DL Project\dlenv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [5]:
"""
This script is for word embeding and majority voting

"""
df1 = pd.read_csv('Audioset\Eval.csv')
df2 = pd.read_csv('Audioset\Train.csv')
# Concatenate the two DataFrames
df = pd.concat([df1, df2], ignore_index=True)
# Step 2: Prepare data for clustering
labels = []
file_names = []
for index, row in df.iterrows():
    label_names = row['Labels'].split(',')  # Split label_names by comma
    file_name = row['Filename']
    for label in label_names:
        labels.append(label.strip())  # Remove leading/trailing whitespaces
        file_names.append(file_name)

# Step 3: Generate Embeddings for each word
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
embeddings = model.encode(labels)
np.save('Audioset\embeddings.npy', embeddings)


<>:5: SyntaxWarning: invalid escape sequence '\E'
<>:6: SyntaxWarning: invalid escape sequence '\T'
<>:5: SyntaxWarning: invalid escape sequence '\E'
<>:6: SyntaxWarning: invalid escape sequence '\T'
C:\Users\giorg\AppData\Local\Temp\ipykernel_18828\1434861830.py:5: SyntaxWarning: invalid escape sequence '\E'
  df1 = pd.read_csv('Audioset\Eval.csv')
C:\Users\giorg\AppData\Local\Temp\ipykernel_18828\1434861830.py:6: SyntaxWarning: invalid escape sequence '\T'
  df2 = pd.read_csv('Audioset\Train.csv')
c:\Users\giorg\OneDrive\Υπολογιστής\DL Project\dlenv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [38]:
""" 

define clusters here maybe we need alitle tuning but the results are  lets say good

"""
embeddings = np.load('Audioset\embeddings.npy')
# Step 4: Cluster the Embeddings using KMeans
n_clusters = 10 # Number of clusters
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
clusters = kmeans.fit_predict(embeddings)

# Step 5: Find the majority cluster for each file
file_cluster = {}
for file_name, cluster in zip(file_names, clusters):
    if file_name not in file_cluster:
        file_cluster[file_name] = {}
    if cluster not in file_cluster[file_name]:
        file_cluster[file_name][cluster] = 0
    file_cluster[file_name][cluster] += 1

# Assign each file to the cluster with the majority of labels
file_cluster_majority = {file_name: max(cluster_dict, key=cluster_dict.get) for file_name, cluster_dict in file_cluster.items()}

# Step 6: Create DataFrame with file names and corresponding clusters
file_cluster_df = pd.DataFrame.from_dict(file_cluster_majority, orient='index', columns=['cluster']).reset_index()
file_cluster_df.columns = ['Filename', 'cluster']
# Save the file names and corresponding clusters to a new CSV file
merged_df = pd.merge(df,file_cluster_df, left_on='Filename', right_on='Filename', how='inner')
merged_df = merged_df.sort_values(by='cluster')
merged_df['cluster'] = 'class_' + (merged_df['cluster'] + 1).astype(str)


<>:6: SyntaxWarning: invalid escape sequence '\e'
<>:6: SyntaxWarning: invalid escape sequence '\e'
C:\Users\giorg\AppData\Local\Temp\ipykernel_18828\1430957654.py:6: SyntaxWarning: invalid escape sequence '\e'
  embeddings = np.load('Audioset\embeddings.npy')


In [39]:
cluster_counts = merged_df['cluster'].value_counts()

print("Unique values in 'Cluster' column and their counts:")
print(cluster_counts)

Unique values in 'Cluster' column and their counts:
cluster
class_9     6209
class_7     6016
class_2     5923
class_10    5027
class_4     3439
class_5     2664
class_1     1958
class_6     1662
class_3     1090
class_8      412
Name: count, dtype: int64


In [17]:
df1 = pd.read_csv('Audioset\CSVs\T_Labels.csv')
df2 = pd.read_csv('Audioset\CSVs\E_Labels.csv')
df = pd.read_csv("C:\\Users\\giorg\\Downloads\\categorized_files.csv")

df = df.sort_values(by='Category')

# Join the DataFrames using a common column
join_1 = pd.merge(df1, df, on='Filename', suffixes=('', '_right'))
join_1 = join_1.drop(columns=['Labels_right', 'cluster'])

join_2 = pd.merge(df2, df, on='Filename', suffixes=('', '_right'))
join_2 = join_2.drop(columns=['Labels_right', 'cluster'])

# Save the merged DataFrames to new CSV files
join_1.to_csv(os.path.join('Audioset', 'GPTTrain.csv'), index=False)
join_2.to_csv(os.path.join('Audioset', 'GPTEval.csv'), index=False)

<>:1: SyntaxWarning: invalid escape sequence '\C'
<>:2: SyntaxWarning: invalid escape sequence '\C'
<>:1: SyntaxWarning: invalid escape sequence '\C'
<>:2: SyntaxWarning: invalid escape sequence '\C'
C:\Users\giorg\AppData\Local\Temp\ipykernel_6800\318386497.py:1: SyntaxWarning: invalid escape sequence '\C'
  df1 = pd.read_csv('Audioset\CSVs\T_Labels.csv')
C:\Users\giorg\AppData\Local\Temp\ipykernel_6800\318386497.py:2: SyntaxWarning: invalid escape sequence '\C'
  df2 = pd.read_csv('Audioset\CSVs\E_Labels.csv')


In [3]:
df = pd.read_csv('Audioset\GPTTrain.csv')
cluster_counts = df['Category'].value_counts()

print("Unique values in 'Cluster' column and their counts:")
print(cluster_counts)

Unique values in 'Cluster' column and their counts:
Category
Miscellaneous              5838
Human Sounds               4731
Musical Instruments        3860
Animal Sounds              1247
Environmental Sounds        962
Vehicle Sounds              899
Machine and Tool Sounds     220
Impact Sounds                76
Name: count, dtype: int64


<>:1: SyntaxWarning: invalid escape sequence '\G'
<>:1: SyntaxWarning: invalid escape sequence '\G'
C:\Users\giorg\AppData\Local\Temp\ipykernel_21824\197808817.py:1: SyntaxWarning: invalid escape sequence '\G'
  df = pd.read_csv('Audioset\GPTTrain.csv')


In [4]:
df = pd.read_csv('Audioset\GPTEval.csv')
cluster_counts = df['Category'].value_counts()

print("Unique values in 'Cluster' column and their counts:")
print(cluster_counts)

Unique values in 'Cluster' column and their counts:
Category
Miscellaneous              5314
Human Sounds               4380
Musical Instruments        3493
Animal Sounds              1230
Environmental Sounds        981
Vehicle Sounds              859
Machine and Tool Sounds     238
Impact Sounds                72
Name: count, dtype: int64


<>:1: SyntaxWarning: invalid escape sequence '\G'
<>:1: SyntaxWarning: invalid escape sequence '\G'
C:\Users\giorg\AppData\Local\Temp\ipykernel_21824\4086195539.py:1: SyntaxWarning: invalid escape sequence '\G'
  df = pd.read_csv('Audioset\GPTEval.csv')
